In [20]:
from copy import deepcopy
from pathlib import Path
import cv2
import matplotlib.pylab as plt
import numpy as np
from scipy.io import loadmat
from scipy.ndimage import gaussian_filter
from tqdm import tqdm
import cv2
import pandas as pd
import numpy as np 
import cv2 
from matplotlib import pyplot as plt 

the dataframes come from IDMatcher-PerFrame.ipynb

All methodes use 1 epipolar geometry made using a random frame.

for df_preds per frame we created a epipolar line per left id and chose the known right id with the lowest euclidean distance to this epipolor line as the "matching" id. This results in certain right_ids occuring more than once in frames. 

For one measure we Then chose the most commonly predicted right id among all frames combined for each left id as its most likely correct match.

For the second measure we calculated the correctness per frame and averaged this out.

For pos_match we took all assumed matches of each left id amongs all frames, created a top three most common ones, calculated the average difference using the difference when predicting from left to right and right to left combined from the predicted points to the "chosen" true points. Used this to determen from the top 3 which ones was the "best" match per id based on the lowest average distance which resulted in result_df. This resulted in some right_ids occuring more than once tho. 

In [113]:
df_preds = pd.read_csv('df_preds_v2.csv')
df_predsV4 = pd.read_csv('df_preds_V5.csv')
df_preds_RE_ID = pd.read_csv('df_preds_Re_ID.csv')
df_pre_RE_ID_129_1 = pd.read_csv('df_pre_RE_ID_129_1.csv')
df_pre_RE_ID_129_2 = pd.read_csv('df_pre_RE_ID_129_2.csv')
df_pre_RE_ID_349_2 = pd.read_csv('df_pre_RE_ID_349_2.csv')
df_pre_RE_ID_349_1 = pd.read_csv('df_pre_RE_ID_349_1.csv')
df_pre_RE_ID_161_1 = pd.read_csv('df_pre_RE_ID_161_1.csv')
df_pre_RE_ID_161_2 = pd.read_csv('df_pre_RE_ID_161_2.csv')
df_pre_RE_ID_183_1 = pd.read_csv('df_pre_RE_ID_183_1.csv')
df_pre_RE_ID_183_2 = pd.read_csv('df_pre_RE_ID_183_2.csv')
df_pre_RE_ID_231_1 = pd.read_csv('df_pre_RE_ID_231_1.csv')
df_pre_RE_ID_231_2 = pd.read_csv('df_pre_RE_ID_231_2.csv')
df_pre_RE_ID_406_1 = pd.read_csv('df_pre_RE_ID_406_1.csv')
df_pre_RE_ID_406_2 = pd.read_csv('df_pre_RE_ID_406_2.csv')
pos_match = pd.read_csv('pos_match_v2.csv')
result_df = pd.read_csv('result_df_v2.csv')
csv_path_1 = '129_1_clean.txt'
df1 = pd.read_csv(csv_path_1)
csv_path_2 = '129_2_clean.txt'
df2 = pd.read_csv(csv_path_2)

In [107]:
# gt_matches = {3:0, 5:1, 4:2, 2:3, 8:4, 0:5, 1:6, 6:7, 7:8, 9:9, 10:10, 11:11, 12:12, 13:13, 14:14}
# gt_matches_Re_ID = {2:8, 4:0, 0:1, 3:6, 6:2, 7:4, 5:7, 8:5, 9:9, 14:13, 10:10, 12:12, 15:14, 11:11, 1:3}
# display(df_preds)
# display(df_predsV4)
# # display(pos_match)
# display(result_df)
# display(df_preds_RE_ID)
# display(df_pre_RE_ID_129_1)
# display(df_pre_RE_ID_349_2)
display(df_pre_RE_ID_231_2)

,tr_id,gt_id,difference,frame
0,0.0,16.0,0.833342,1
1,1.0,1.0,3.236084,1
2,2.0,2.0,1.938984,1
3,3.0,4.0,1.459671,1
4,4.0,5.0,3.252842,1
...,...,...,...,...
82921,21.0,8.0,0.448199,3116
82922,40.0,14.0,0.899174,3116
82923,41.0,34.0,1.802897,3116
82924,43.0,27.0,3.578217,3116


In [117]:
# df_pre_RE_ID


# # Group by 'left_id' and count occurrences of each 'right_id'
# most_common_tr_gt_129_1 = df_pre_RE_ID_129_1.groupby('tr_id')['gt_id'].agg(lambda x: x.value_counts().index[0])

# # Convert Series to DataFrame
# most_common_tr_gt_129_1 = most_common_tr_gt_129_1.reset_index(name='gt_id')

# display(most_common_tr_gt_129_1)

# gt_id_to_tr_id_REAL_129_1 = {1:0,3:1, 9:2, 2:6, 7:3, 8:5, 4:7, 5:8, 6:4, 14:14, 15:15, 13:12, 12:11, 11:10, 10:9}

# print(gt_id_to_tr_id_REAL_129_1)

# # id matcher on ground truth to predicted on 129_1 gives a matches dict with accuracy close to 
# # real matches aquired from watching video frame by frame 
# # 14/15 good


# # Group by 'left_id' and count occurrences of each 'right_id'
# most_common_tr_gt_129_2 = df_pre_RE_ID_129_2.groupby('tr_id')['gt_id'].agg(lambda x: x.value_counts().index[0])

# # Convert Series to DataFrame
# most_common_tr_gt_129_2 = most_common_tr_gt_129_2.reset_index(name='gt_id')

# display(most_common_tr_gt_129_2)

# gt_id_to_tr_id_REAL_129_2 = {5:8, 8:6, 9:7, 6:1, 7:2, 4:3, 2:0, 1:4, 3:5, 11:10, 12:11, 10:9, 14:13, 13:12, 15:14}

# print(gt_id_to_tr_id_REAL_129_2)



# # Group by 'left_id' and count occurrences of each 'right_id'
# most_common_tr_gt_349_2 = df_pre_RE_ID_349_2.groupby('tr_id')['gt_id'].agg(lambda x: x.value_counts().index[0])

# # Convert Series to DataFrame
# most_common_tr_gt_349_2 = most_common_tr_gt_349_2.reset_index(name='gt_id')

# display(most_common_tr_gt_349_2)

# gt_id_to_tr_id_REAL_349_2 = {1:2, 7:0, 6:7, 2:6, 8:4, 4:1, 3:5, 5:3, 9:8}

# print(gt_id_to_tr_id_REAL_349_2)



# # Group by 'left_id' and count occurrences of each 'right_id'
# most_common_tr_gt_349_1 = df_pre_RE_ID_349_1.groupby('tr_id')['gt_id'].agg(lambda x: x.value_counts().index[0])

# # Convert Series to DataFrame
# most_common_tr_gt_349_1 = most_common_tr_gt_349_1.reset_index(name='gt_id')

# display(most_common_tr_gt_349_1)

# gt_id_to_tr_id_REAL_349_1 = {1:0, 2:1, 7:5, 3:3, 5:4, 4:6, 6:2, 9:8, 10:9, 8:7}

# print(gt_id_to_tr_id_REAL_349_1)


# Group by 'left_id' and count occurrences of each 'right_id'
# most_common_tr_gt_161_1 = df_pre_RE_ID_161_1.groupby('tr_id')['gt_id'].agg(lambda x: x.value_counts().index[0])

# # Convert Series to DataFrame
# most_common_tr_gt_161_1 = most_common_tr_gt_161_1.reset_index(name='gt_id')

# display(most_common_tr_gt_161_1)

# gt_id_to_tr_id_REAL_161_1 = {6:1, 8:2, 9:7, 1:5, 7:10, 4:5, 12:9, 11:3, 3:4, 11:3, 10:11, 2:0, 13:12, 5:8, 14:13, 15:15, 16:16, 17:19, 18:20, 19:21, 20:22, 21:23}

# print(gt_id_to_tr_id_REAL_161_1)


# # Group by 'left_id' and count occurrences of each 'right_id'
# most_common_tr_gt_161_2 = df_pre_RE_ID_161_2.groupby('tr_id')['gt_id'].agg(lambda x: x.value_counts().index[0])

# # Convert Series to DataFrame
# most_common_tr_gt_161_2 = most_common_tr_gt_161_2.reset_index(name='gt_id')

# display(most_common_tr_gt_161_2)

# gt_id_to_tr_id_REAL_161_2 = {1:0,2:3,3:7,4:8,5:6,6:10,7:2,8:4,9:5,10:1,11:11,12:9,13:14,14:12,15:15,16:17,17:5,18:5
#                              ,19:25,20:30,21:33,22:34,23:36}

# print(gt_id_to_tr_id_REAL_161_2)

# # Group by 'left_id' and count occurrences of each 'right_id'
# most_common_tr_gt_183_1 = df_pre_RE_ID_183_1.groupby('tr_id')['gt_id'].agg(lambda x: x.value_counts().index[0])

# # Convert Series to DataFrame
# most_common_tr_gt_183_1 = most_common_tr_gt_183_1.reset_index(name='gt_id')

# display(most_common_tr_gt_183_1)

# gt_id_to_tr_id_REAL_183_1 = {1:0,2:3,3:7,4:8,5:6,6:10,7:2,8:4,9:5,10:1,11:11,12:9,13:14,14:12,15:15,16:17,17:5,18:5
#                              ,19:25,20:30,21:33,22:34,23:36}

# print(gt_id_to_tr_id_REAL_183_1)

# # Group by 'left_id' and count occurrences of each 'right_id'
# most_common_tr_gt_231_1 = df_pre_RE_ID_231_1.groupby('tr_id')['gt_id'].agg(lambda x: x.value_counts().index[0])

# # Convert Series to DataFrame
# most_common_tr_gt_231_1 = most_common_tr_gt_231_1.reset_index(name='gt_id')

# display(most_common_tr_gt_231_1)

# # Group by 'left_id' and count occurrences of each 'right_id'
# most_common_tr_gt_231_2 = df_pre_RE_ID_231_2.groupby('tr_id')['gt_id'].agg(lambda x: x.value_counts().index[0])

# # Convert Series to DataFrame
# most_common_tr_gt_231_2 = most_common_tr_gt_231_2.reset_index(name='gt_id')

# display(most_common_tr_gt_231_2)

# # Group by 'left_id' and count occurrences of each 'right_id'
# most_common_tr_gt_406_1 = df_pre_RE_ID_406_1.groupby('tr_id')['gt_id'].agg(lambda x: x.value_counts().index[0])

# # Convert Series to DataFrame
# most_common_tr_gt_406_1 = most_common_tr_gt_406_1.reset_index(name='gt_id')

# display(most_common_tr_gt_406_1)

# Group by 'left_id' and count occurrences of each 'right_id'
most_common_tr_gt_406_2 = df_pre_RE_ID_406_2.groupby('tr_id')['gt_id'].agg(lambda x: x.value_counts().index[0])

# Convert Series to DataFrame
most_common_tr_gt_406_2 = most_common_tr_gt_406_2.reset_index(name='gt_id')

display(most_common_tr_gt_406_2)

,tr_id,gt_id
0,0.0,4.0
1,1.0,6.0
2,2.0,8.0
3,3.0,1.0
4,4.0,5.0
5,5.0,2.0
6,6.0,3.0
7,7.0,7.0
8,8.0,9.0
9,9.0,5.0


In [133]:
import pandas as pd

# Assuming df is your DataFrame containing the 'tr_id' and 'gt_id' columns

# Count the occurrences of each gt_id
gt_id_counts = most_common_tr_gt_183_1['gt_id'].value_counts()

# Filter gt_id values that appear more than once
duplicate_gt_ids = gt_id_counts[gt_id_counts > 1].index.tolist()

print(duplicate_gt_ids,len(duplicate_gt_ids))

print(set(most_common_tr_gt_183_1['gt_id']))
most_common_tr_gt_183_1

[31.0, 32.0, 23.0, 27.0, 24.0, 8.0, 30.0, 28.0, 29.0, 25.0, 11.0, 10.0, 7.0, 22.0, 14.0, 20.0, 17.0, 21.0, 9.0, 26.0, 42.0, 37.0, 36.0, 45.0, 4.0, 16.0, 15.0, 19.0, 12.0, 5.0, 6.0, 3.0, 2.0, 39.0] 34
{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0}


,tr_id,gt_id
0,0.0,13.0
1,1.0,19.0
2,2.0,17.0
3,3.0,21.0
4,4.0,1.0
...,...,...
144,144.0,22.0
145,145.0,45.0
146,146.0,27.0
147,147.0,8.0


In [110]:
# import pandas as pd

# # Your DataFrame
# df_data = most_common_tr_gt_161_2.copy()

# df = pd.DataFrame(df_data)

# # Your dictionary
# dict_data = gt_id_to_tr_id_REAL_161_2
# # Initialize lists to store results
# pairs_in_df = []
# keys_with_multiple_values = []

# # Check if key-value pairs exist in the DataFrame and track keys with multiple values
# for key, value in dict_data.items():
#     if (df['gt_id'] == key).any() and (df['tr_id'] == value).any():
#         pairs_in_df.append((key, value))
#     if df['gt_id'].value_counts().get(key, 0) > 1:
#         keys_with_multiple_values.append(key)

# print("Key-value pairs in DataFrame:")
# print(pairs_in_df,len(pairs_in_df),len(most_common_tr_gt_161_2),len(gt_id_to_tr_id_REAL_161_2))
# print("\nKeys with multiple values in DataFrame:")
# print(keys_with_multiple_values,len(keys_with_multiple_values))


In [111]:
# # Initialize a counter for multiple values not in the dictionary
# multiple_values_not_in_dict = 0

# # Iterate through keys with multiple values
# for key in keys_with_multiple_values:
#     # Find indices where the key appears in the DataFrame
#     indices = df.index[df['gt_id'] == key]
    
#     # Get corresponding values for the key
#     values = df.loc[indices, 'tr_id'].tolist()
    
#     # Count occurrences not in the dictionary
#     for value in values:
#         if value not in dict_data.values():
#             multiple_values_not_in_dict += 1

# print("Number of multiple values not in the dictionary:", multiple_values_not_in_dict)


In [119]:
# most_common_tr_gt_129_1.to_csv('most_common_tr_gt_129_1.csv', index=False)
# most_common_tr_gt_129_2.to_csv('most_common_tr_gt_129_2.csv', index=False)
# most_common_tr_gt_349_1.to_csv('most_common_tr_gt_349_1.csv', index=False)
most_common_tr_gt_406_2.to_csv('most_common_tr_gt_406_2.csv', index=False)

In [22]:
# Group by 'left_id' and count occurrences of each 'right_id'
most_common_right_id = df_preds.groupby('left_id')['right_id'].agg(lambda x: x.value_counts().index[0])

# Convert Series to DataFrame
most_common_df = most_common_right_id.reset_index(name='right_id')

most_common_df

,left_id,right_id
0,0.0,5.0
1,1.0,6.0
2,2.0,3.0
3,3.0,0.0
4,4.0,2.0
5,5.0,1.0
6,6.0,7.0
7,7.0,9.0
8,8.0,4.0
9,9.0,9.0


In [32]:
# subset of 298 frames version with epipolar per frame

# Group by 'left_id' and count occurrences of each 'right_id'
most_common_right_id_v2 = df_predsV4.groupby('left_id')['right_id'].agg(lambda x: x.value_counts().index[0])

# Convert Series to DataFrame
most_common_df_v2 = most_common_right_id_v2.reset_index(name='right_id')

display(most_common_df_v2)


# Group by 'left_id' and count occurrences of each 'right_id'
most_common_right_id_v4 = df_preds_RE_ID.groupby('left_id')['right_id'].agg(lambda x: x.value_counts().index[0])

# Convert Series to DataFrame
most_common_df_v4 = most_common_right_id_v4.reset_index(name='right_id')




# subset of 298 frames version with epipolar once

# df_preds_normal_for_comp = df_preds.loc[df_preds['frame'] < 298]

# # Group by 'left_id' and count occurrences of each 'right_id'
# most_common_right_id_v3 = df_preds_normal_for_comp.groupby('left_id')['right_id'].agg(lambda x: x.value_counts().index[0])

# # Convert Series to DataFrame
# most_common_df_v3 = most_common_right_id_v3.reset_index(name='right_id')

# display(most_common_df_v3)



df_preds_normal_for_comp = df_preds

# Group by 'left_id' and count occurrences of each 'right_id'
most_common_right_id_v3 = df_preds_normal_for_comp.groupby('left_id')['right_id'].agg(lambda x: x.value_counts().index[0])

# Convert Series to DataFrame
most_common_df_v3 = most_common_right_id_v3.reset_index(name='right_id')

display(most_common_df_v3)

print("V4")
display(most_common_df_v4)


def performance_matches(df,matches):
    # based on most common matches across all frames
    matches_count_m1 = 0
    for x in range(len(df)):
        l = df['left_id'][x]
        r = df['right_id'][x]
        if matches[l] == r:
            matches_count_m1 += 1
    print(matches_count_m1/len(df))
    
    
    
def performance_matches_v2(df,matches):
    # based on most common matches across all frames
    matches_count_m1 = 0
    for x in range(len(df)):
        l = df['left_id'][x]
        r = df['right_id'][x]
        if l in matches:
            if matches[l] == r:
                matches_count_m1 += 1
    print(matches_count_m1/len(df))
    
    
performance_matches(most_common_df_v3,gt_matches)
performance_matches(most_common_df_v2,gt_matches)
performance_matches_v2(most_common_df_v4,gt_matches_Re_ID)
print(gt_matches)

,left_id,right_id
0,0.0,5.0
1,1.0,6.0
2,2.0,3.0
3,3.0,0.0
4,4.0,2.0
5,5.0,1.0
6,6.0,7.0
7,7.0,8.0
8,8.0,4.0
9,9.0,9.0


,left_id,right_id
0,0.0,5.0
1,1.0,6.0
2,2.0,3.0
3,3.0,0.0
4,4.0,2.0
5,5.0,1.0
6,6.0,7.0
7,7.0,9.0
8,8.0,4.0
9,9.0,9.0


V4


,left_id,right_id
0,0.0,1.0
1,1.0,3.0
2,2.0,8.0
3,3.0,6.0
4,4.0,0.0
5,5.0,7.0
6,6.0,2.0
7,7.0,4.0
8,8.0,5.0
9,9.0,9.0


0.8
0.9333333333333333
0.875
{3: 0, 5: 1, 4: 2, 2: 3, 8: 4, 0: 5, 1: 6, 6: 7, 7: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14}


performance is better when doing epipolar per frame, difference of 13.33% 

In [7]:
print(gt_matches)

{3: 0, 5: 1, 4: 2, 2: 3, 8: 4, 0: 5, 1: 6, 6: 7, 7: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14}


The method below calculates corectness by picking the most commonly predicted right id per left id and setting that as a match.

In [8]:
# based on most common matches across all frames
matches_count_m1 = 0
for x in range(len(most_common_df)):
    l = most_common_df['left_id'][x]
    r =  most_common_df['right_id'][x]
    if gt_matches[l] == r:
        matches_count_m1 += 1
        
print(matches_count_m1/len(most_common_df))

0.8


In [9]:
result_df

,left_right,left_boy,right_boy,avg_diff,left_right_diff,right_left_diff
0,"(1.0, 8.0)",0.0,7.0,39.212002,31.038243,8.173759
1,"(10.0, 10.0)",9.0,9.0,16.562931,6.784330,9.778601
2,"(11.0, 1.0)",10.0,0.0,6.069236,2.411325,3.657911
3,"(12.0, 2.0)",11.0,1.0,10.654859,1.745077,8.909782
4,"(13.0, 13.0)",12.0,12.0,12.415106,4.173788,8.241318
5,"(14.0, 14.0)",13.0,13.0,52.197162,2.936132,49.261030
6,"(15.0, 15.0)",14.0,14.0,66.361335,12.157227,54.204109
7,"(2.0, 7.0)",1.0,6.0,55.052347,28.305855,26.746493
8,"(3.0, 4.0)",2.0,3.0,38.985178,3.520585,35.464594
9,"(4.0, 12.0)",3.0,11.0,7.091647,2.568191,4.523456


the method below calculates correctnes using the matches from result_df

In [10]:
l_boys = list(result_df['left_boy'])
r_boys = list(result_df['right_boy'])

df_avg_dif_based = pd.DataFrame({'left_id': l_boys, 'right_id': r_boys})
display(df_avg_dif_based)
# based on best matches based on lowest average distance from predicted match to true match
matches_count_m2 = 0
for x in range(len(df_avg_dif_based )):
    l = df_avg_dif_based ['left_id'][x]
    r =  df_avg_dif_based ['right_id'][x]
    if gt_matches[l] == r:
        print(l,r)
        matches_count_m2 += 1
        
print(matches_count_m2/len(df_avg_dif_based))
print(gt_matches)

,left_id,right_id
0,0.0,7.0
1,9.0,9.0
2,10.0,0.0
3,11.0,1.0
4,12.0,12.0
5,13.0,13.0
6,14.0,14.0
7,1.0,6.0
8,2.0,3.0
9,3.0,11.0


9.0 9.0
12.0 12.0
13.0 13.0
14.0 14.0
1.0 6.0
2.0 3.0
4.0 2.0
0.4666666666666667
{3: 0, 5: 1, 4: 2, 2: 3, 8: 4, 0: 5, 1: 6, 6: 7, 7: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14}


The method below calculates correctness of matches per frame. then averages this out amongst all frames.

In [11]:
# based on amount of matches per frame
match_count_dict = dict()
for frame in set(df_preds['frame']):
    match_count = 0
    temp = df_preds.loc[df_preds['frame'] == frame]
#     print(list(temp['left_id']),list(temp['right_id']))
    left_l = list(temp['left_id'])
    right_l = list(temp['right_id'])
    for x in range(len(left_l)):
#         print(left_l[x],right_l[x])
        if gt_matches[left_l[x]] == right_l[x]:
#             print("MATCH",left_l[x],right_l[x],frame)
            match_count += 1
    match_count_dict[frame] = match_count/len(left_l)
# match_count_dict

average_value_matches = sum(match_count_dict.values()) / len(match_count_dict)
print(average_value_matches)
# print(match_count_dict)

0.667581481007858


In [24]:
# RE-ID
# based on amount of matches per frame
match_count_dict = dict()
for frame in set(df_preds_RE_ID['frame']):
    match_count = 0
    temp = df_preds_RE_ID.loc[df_preds_RE_ID['frame'] == frame]
#     print(list(temp['left_id']),list(temp['right_id']))
    left_l = list(temp['left_id'])
    right_l = list(temp['right_id'])
    for x in range(len(left_l)):
#         print(left_l[x],right_l[x])
#         print(x,left_l[x],right_l[x])
        if (left_l[x] != 15.0) & (right_l[x] != 15.0):
            if gt_matches[left_l[x]] == right_l[x]:
    #             print("MATCH",left_l[x],right_l[x],frame)
                match_count += 1
    match_count_dict[frame] = match_count/len(left_l)
# match_count_dict

average_value_matches = sum(match_count_dict.values()) / len(match_count_dict)
print(average_value_matches)
# print(match_count_dict)

0.08774498146490463


In [12]:
# V2 with epipolar per frame
df_predsV4 = pd.read_csv('df_preds_V5.csv')
df_preds_normal_for_comp = df_preds
display(df_preds_normal_for_comp)
display(df_predsV4)

def measure_correctness(df,matches):
    match_count_dict = dict()
    for frame in set(df['frame']):
        match_count = 0
        temp = df.loc[df['frame'] == frame]
    #     print(list(temp['left_id']),list(temp['right_id']))
        left_l = list(temp['left_id'])
        right_l = list(temp['right_id'])
        for x in range(len(left_l)):
    #         print(left_l[x],right_l[x])
            if matches[left_l[x]] == right_l[x]:
    #             print("MATCH",left_l[x],right_l[x],frame)
                match_count += 1
        match_count_dict[frame] = match_count/len(left_l)
    # match_count_dict
    average_value_matches = sum(match_count_dict.values()) / len(match_count_dict)
    print(average_value_matches)
    
measure_correctness(df_preds_normal_for_comp,gt_matches)
measure_correctness(df_predsV4,gt_matches)

,left_id,right_id,difference,frame
0,0.0,5.0,27.999137,1.0
1,1.0,5.0,4.070687,1.0
2,2.0,3.0,2.869824,1.0
3,3.0,0.0,4.092777,1.0
4,4.0,2.0,6.089725,1.0
...,...,...,...,...
30196,4.0,10.0,3.773182,3117.0
30197,5.0,1.0,12.651285,3117.0
30198,6.0,7.0,10.728657,3117.0
30199,7.0,10.0,3.773182,3117.0


,left_id,right_id,difference,frame
0,0.0,5.0,5.930170,1.0
1,1.0,5.0,5.930170,1.0
2,2.0,3.0,7.923938,1.0
3,3.0,0.0,5.477780,1.0
4,4.0,2.0,6.025641,1.0
...,...,...,...,...
30157,4.0,8.0,2.652197,3114.0
30158,5.0,1.0,10.625984,3114.0
30159,6.0,7.0,5.238354,3114.0
30160,7.0,8.0,2.652197,3114.0


0.667581481007858
0.6745719633966348


when using epipolar on each frame it performs better then when using epipolar once on a random frame, on the subset of 298 frames there is a performance difference of less then 1%. 

In [47]:
display(df_preds)
display(df1)
display(df2)

,left_id,right_id,difference,frame
0,0.0,5.0,27.999137,1.0
1,1.0,5.0,4.070687,1.0
2,2.0,3.0,2.869824,1.0
3,3.0,0.0,4.092777,1.0
4,4.0,2.0,6.089725,1.0
...,...,...,...,...
30196,4.0,10.0,3.773182,3117.0
30197,5.0,1.0,12.651285,3117.0
30198,6.0,7.0,10.728657,3117.0
30199,7.0,10.0,3.773182,3117.0


,frame,id,x,y,x_offset,y_offset
0,1,0,1212.98,416.74,26.04,14.37
1,1,1,1151.14,415.00,27.58,10.34
2,1,2,1487.32,431.94,17.38,19.86
3,1,3,1080.11,337.38,18.68,6.46
4,1,4,1220.90,325.30,21.28,10.54
...,...,...,...,...,...,...
30196,3117,4,1011.60,235.99,10.38,22.01
30197,3117,5,1304.99,217.40,15.01,24.80
30198,3117,6,969.60,489.40,23.90,16.40
30199,3117,7,1037.36,245.93,14.89,29.79


,frame,id,x,y,x_offset,y_offset
0,1,0,1010.39,357.55,21.50,7.74
1,1,1,1151.46,337.71,20.11,17.63
2,1,2,1173.87,341.47,11.20,12.02
3,1,3,1446.30,460.60,19.11,20.90
4,1,4,1066.87,626.29,48.07,6.87
...,...,...,...,...,...,...
30644,3118,4,949.07,696.08,23.25,33.30
30645,3118,5,1067.44,288.15,38.96,21.85
30646,3118,6,1188.06,193.33,33.94,17.14
30647,3118,7,883.91,514.11,25.29,14.69


In [16]:
# Load necessary matrices and coefficients
dd = loadmat("stereoParams_Dep4.mat")
distCoeffs1 = deepcopy(dd["distortionCoefficients1"])
distCoeffs2 = deepcopy(dd["distortionCoefficients2"])
cameraMatrix1 = deepcopy(dd["intrinsicMatrix1"])
cameraMatrix2 = deepcopy(dd["intrinsicMatrix2"])
R = deepcopy(dd["rotationOfCamera2"])
T = deepcopy(dd["translationOfCamera2"])
cameraMatrix1[0:2, 2] += 1
cameraMatrix2[0:2, 2] += 1
# Projection matrices
P1 = np.dot(cameraMatrix1, np.hstack((np.eye(3), np.zeros((3, 1)))))
P2 = np.dot(cameraMatrix2, np.hstack((R, T.reshape(3, 1))))
# df_preds_test = df_preds.loc[df_preds['frame'] == 1]
# df_preds_test

uses df_preds dataframe and matches which score on average 0.67 accuracy

In [22]:
df_3d = pd.DataFrame()
for index, row in df_preds.iterrows():
#     print(row['left_id'])
    df1_temp = df1.loc[(df1['id'] == row['left_id']) & (df1['frame'] == row['frame'])]
    df2_temp = df2.loc[(df2['id'] == row['right_id']) & (df2['frame'] == row['frame'])]
    left_x = float(df1_temp['x'])
    left_y = float(df1_temp['y'])
    left_cor = np.array([left_x,left_y])
    right_x = float(df2_temp['x'])
    right_y = float(df2_temp['y'])
    right_cor = np.array([right_x,right_y])
    left_id = int(df1_temp['id'])
    right_id = int(df2_temp['id'])
#     print(int(row['left_id']),left_id,left_cor,int(row['right_id']),right_id,right_cor)
    # # Undistort points
    undistortedPoints1 = cv2.undistortPoints(
        left_cor, cameraMatrix1, distCoeffs1, P=cameraMatrix1
    )  # result Nx1x2. No need for np.expand_dims(imagePoints1, axis=1)
    undistortedPoints2 = cv2.undistortPoints(
        right_cor, cameraMatrix2, distCoeffs2, P=cameraMatrix2
    )
    
    # Perform triangulation
    point_3d_homogeneous = cv2.triangulatePoints(P1, P2, undistortedPoints1, undistortedPoints2)
    # Convert homogeneous coordinates to Cartesian
    point_3d_cartesian = point_3d_homogeneous[:3] / point_3d_homogeneous[3]
    # Extract x, y, z coordinates
    x, y, z = point_3d_cartesian.flatten()
    
#     new_row_data = {'left_id': left_id, 'right_id': row['id'],'difference': difference,'frame':row['frame']}
#     new_row = pd.DataFrame(new_row_data,index=[0])
    
    df_temp = pd.DataFrame({'frame':int(row['frame']),'left_id': left_id, 'right_id': right_id,'x': x,'y': y, 'z': z},index=[0])
    df_3d = pd.concat([df_3d, df_temp], ignore_index=True)
    
df_3d

/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_2021/1338928614.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  left_x = float(df1_temp['x'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_2021/1338928614.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  left_y = float(df1_temp['y'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_2021/1338928614.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  right_x = float(df2_temp['x'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_2021/1338928614.py:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  righ

,frame,left_id,right_id,x,y,z
0,1,0,5,475.641316,-251.609266,2175.999486
1,1,1,5,478.872969,-333.698692,2894.645160
2,1,2,3,1187.891829,-270.103752,2587.299296
3,1,3,0,273.150068,-478.047934,2606.051411
4,1,4,2,634.803972,-551.119175,2809.724279
...,...,...,...,...,...,...
30196,3117,4,10,153.236529,-871.108718,3291.131236
30197,3117,5,1,770.093280,-740.882905,2565.283206
30198,3117,6,7,26.176099,-131.034777,2496.313876
30199,3117,7,10,195.927570,-744.434856,2857.989353


In [23]:
df_3d['x'].min()

-23186.503588355223

In [24]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display
df = df_3d
# Group the DataFrame by 'frame'
grouped = df.groupby('frame')

# Create a function to update the plot based on the selected frame
def update_plot(frame):
    data = grouped.get_group(frame)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(data['x'], data['y'], data['z'], marker='o')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'Frame {frame}')
    plt.show()

# Create a slider widget
frame_slider = widgets.IntSlider(min=df['frame'].min(), max=df['frame'].max(), value=df['frame'].min(), description='Frame')

# Display the slider and plot
widgets.interact(update_plot, frame=frame_slider)


interactive(children=(IntSlider(value=1, description='Frame', max=3117, min=1), Output()), _dom_classes=('widg…

<function __main__.update_plot(frame)>

uses most_common_df which uses most common match among all frames for matches and results in 0.8 accuracy

In [13]:
display(most_common_df)
most_common_dict = dict(zip(most_common_df['left_id'], most_common_df['right_id']))
print(most_common_dict)


display(most_common_df_v2)
most_common_dict_v2 = dict(zip(most_common_df_v2['left_id'],most_common_df_v2['right_id']))
print(most_common_dict_v2)



,left_id,right_id
0,0.0,5.0
1,1.0,6.0
2,2.0,3.0
3,3.0,0.0
4,4.0,2.0
5,5.0,1.0
6,6.0,7.0
7,7.0,9.0
8,8.0,4.0
9,9.0,9.0


{0.0: 5.0, 1.0: 6.0, 2.0: 3.0, 3.0: 0.0, 4.0: 2.0, 5.0: 1.0, 6.0: 7.0, 7.0: 9.0, 8.0: 4.0, 9.0: 9.0, 10.0: 10.0, 11.0: 11.0, 12.0: 1.0, 13.0: 13.0, 14.0: 13.0}


,left_id,right_id
0,0.0,5.0
1,1.0,6.0
2,2.0,3.0
3,3.0,0.0
4,4.0,2.0
5,5.0,1.0
6,6.0,7.0
7,7.0,8.0
8,8.0,4.0
9,9.0,9.0


{0.0: 5.0, 1.0: 6.0, 2.0: 3.0, 3.0: 0.0, 4.0: 2.0, 5.0: 1.0, 6.0: 7.0, 7.0: 8.0, 8.0: 4.0, 9.0: 9.0, 10.0: 10.0, 11.0: 11.0, 12.0: 12.0, 13.0: 13.0, 14.0: 13.0}


In [46]:
csv_path_1 = '129_1_clean.txt'
df1 = pd.read_csv(csv_path_1)
csv_path_2 = '129_2_clean.txt'
df2 = pd.read_csv(csv_path_2)
display(df1)
display(df2)
df1['id'] = df1['id'] - 1
df2['id'] = df2['id'] - 1
display(df1)
display(df2)

,frame,id,x,y,x_offset,y_offset
0,1,1,1212.98,416.74,26.04,14.37
1,1,2,1151.14,415.00,27.58,10.34
2,1,3,1487.32,431.94,17.38,19.86
3,1,4,1080.11,337.38,18.68,6.46
4,1,5,1220.90,325.30,21.28,10.54
...,...,...,...,...,...,...
30196,3117,5,1011.60,235.99,10.38,22.01
30197,3117,6,1304.99,217.40,15.01,24.80
30198,3117,7,969.60,489.40,23.90,16.40
30199,3117,8,1037.36,245.93,14.89,29.79


,frame,id,x,y,x_offset,y_offset
0,1,1,1010.39,357.55,21.50,7.74
1,1,2,1151.46,337.71,20.11,17.63
2,1,3,1173.87,341.47,11.20,12.02
3,1,4,1446.30,460.60,19.11,20.90
4,1,5,1066.87,626.29,48.07,6.87
...,...,...,...,...,...,...
30644,3118,5,949.07,696.08,23.25,33.30
30645,3118,6,1067.44,288.15,38.96,21.85
30646,3118,7,1188.06,193.33,33.94,17.14
30647,3118,8,883.91,514.11,25.29,14.69


,frame,id,x,y,x_offset,y_offset
0,1,0,1212.98,416.74,26.04,14.37
1,1,1,1151.14,415.00,27.58,10.34
2,1,2,1487.32,431.94,17.38,19.86
3,1,3,1080.11,337.38,18.68,6.46
4,1,4,1220.90,325.30,21.28,10.54
...,...,...,...,...,...,...
30196,3117,4,1011.60,235.99,10.38,22.01
30197,3117,5,1304.99,217.40,15.01,24.80
30198,3117,6,969.60,489.40,23.90,16.40
30199,3117,7,1037.36,245.93,14.89,29.79


,frame,id,x,y,x_offset,y_offset
0,1,0,1010.39,357.55,21.50,7.74
1,1,1,1151.46,337.71,20.11,17.63
2,1,2,1173.87,341.47,11.20,12.02
3,1,3,1446.30,460.60,19.11,20.90
4,1,4,1066.87,626.29,48.07,6.87
...,...,...,...,...,...,...
30644,3118,4,949.07,696.08,23.25,33.30
30645,3118,5,1067.44,288.15,38.96,21.85
30646,3118,6,1188.06,193.33,33.94,17.14
30647,3118,7,883.91,514.11,25.29,14.69


In [27]:
display(df1.loc[df1['frame'] == 2])
display(df2.loc[df2['frame'] == 2])

,frame,id,x,y,x_offset,y_offset
9,2,0,1212.98,416.70,25.70,14.69
10,2,1,1150.88,414.92,27.58,10.35
11,2,2,1487.39,431.99,17.37,19.86
12,2,3,1080.11,337.29,18.68,6.51
13,2,4,1220.85,325.02,21.15,10.62
14,2,5,1211.29,319.44,27.32,9.04
15,2,6,1069.43,499.20,20.92,10.81
16,2,7,1091.36,476.28,17.51,7.76
17,2,8,1190.17,591.21,32.52,9.49


,frame,id,x,y,x_offset,y_offset
9,2,0,1010.38,357.49,21.50,7.74
10,2,1,1151.46,337.71,20.17,17.34
11,2,2,1173.75,341.27,11.20,12.03
12,2,3,1446.41,460.70,19.11,20.90
13,2,4,1066.22,626.24,48.07,6.87
14,2,5,1103.69,442.34,26.32,12.24
15,2,6,1037.62,437.88,37.15,14.25
16,2,7,988.06,528.35,18.49,9.03
17,2,8,978.94,500.22,28.07,8.96


In [14]:
print(most_common_dict)
print(most_common_dict_v2)

{0.0: 5.0, 1.0: 6.0, 2.0: 3.0, 3.0: 0.0, 4.0: 2.0, 5.0: 1.0, 6.0: 7.0, 7.0: 9.0, 8.0: 4.0, 9.0: 9.0, 10.0: 10.0, 11.0: 11.0, 12.0: 1.0, 13.0: 13.0, 14.0: 13.0}
{0.0: 5.0, 1.0: 6.0, 2.0: 3.0, 3.0: 0.0, 4.0: 2.0, 5.0: 1.0, 6.0: 7.0, 7.0: 8.0, 8.0: 4.0, 9.0: 9.0, 10.0: 10.0, 11.0: 11.0, 12.0: 12.0, 13.0: 13.0, 14.0: 13.0}


if a match is an id that is not in the frame (so a incorrect match) then drop both fish.

potential solution: take the frame, do epipolar geometry, make a new best guess for the match from the list of possible matches based on the frame (either exlude all fish that have been matched already for that frame or dont) make this the new match and continue. 

In [29]:
df_3d_v2 = pd.DataFrame()
for frame in set(df1['frame']):
#     print(frame,"frame")
    df1_temp = df1.loc[df1['frame'] == frame]
    df2_temp = df2.loc[df2['frame'] == frame]
    for cur_id in list(df1_temp['id']):
        df1_tt = df1_temp.loc[df1_temp['id'] == cur_id]
#         df2_tt = df2_temp.loc[df2_temp['id'] == cur_id]
        x_1 = float(df1_tt['x'])
        y_1 = float(df1_tt['y'])
        match_id = most_common_dict[cur_id]
#         print(cur_id,"cur",(match_id),"matchID")
        if match_id in list(df2_temp['id']):
#             print(cur_id,match_id, "pair")
            df2_tt = df2_temp.loc[df2_temp['id'] == match_id]
            x_2 = float(df2_tt['x'])
            y_2 = float(df2_tt['y'])
#             print(x_1,y_1,cur_id,l_id_corrected,match_id,x_2,y_2) 
            left_cor = np.array([x_1,y_1])
            right_cor = np.array([x_2,y_2])
            
            undistortedPoints1 = cv2.undistortPoints(left_cor, cameraMatrix1, distCoeffs1, P=cameraMatrix1)  # result Nx1x2. No need for np.expand_dims(imagePoints1, axis=1)
            undistortedPoints2 = cv2.undistortPoints(right_cor, cameraMatrix2, distCoeffs2, P=cameraMatrix2)
    
            # Perform triangulation
            point_3d_homogeneous = cv2.triangulatePoints(P1, P2, undistortedPoints1, undistortedPoints2)
            # Convert homogeneous coordinates to Cartesian
            point_3d_cartesian = point_3d_homogeneous[:3] / point_3d_homogeneous[3]
            # Extract x, y, z coordinates
            x, y, z = point_3d_cartesian.flatten()

        #     new_row_data = {'left_id': left_id, 'right_id': row['id'],'difference': difference,'frame':row['frame']}
        #     new_row = pd.DataFrame(new_row_data,index=[0])

            df_temp = pd.DataFrame({'frame':frame,'left_id': cur_id, 'right_id': match_id,'x': x,'y': y, 'z': z},index=[0])
            df_3d_v2 = pd.concat([df_3d_v2, df_temp], ignore_index=True)
            
display(df_3d_v2)

/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_2021/440496333.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  x_1 = float(df1_tt['x'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_2021/440496333.py:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y_1 = float(df1_tt['y'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_2021/440496333.py:16: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  x_2 = float(df2_tt['x'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_2021/440496333.py:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y_2 = float(df2_tt['y'

,frame,left_id,right_id,x,y,z
0,1,0,5.0,475.641316,-251.609266,2175.999486
1,1,1,6.0,358.941668,-255.299200,2170.809067
2,1,2,3.0,1187.891829,-270.103752,2587.299296
3,1,3,0.0,273.150068,-478.047934,2606.051411
4,1,4,2.0,634.803972,-551.119175,2809.724279
...,...,...,...,...,...,...
28126,3117,3,0.0,108.709105,-315.552335,2472.854547
28127,3117,4,2.0,116.005151,-677.619052,2495.082079
28128,3117,5,1.0,770.093280,-740.882905,2565.283206
28129,3117,6,7.0,26.176099,-131.034777,2496.313876


In [19]:
df_3d_v3 = pd.DataFrame()
for frame in set(df1['frame']):
#     print(frame,"frame")
    df1_temp = df1.loc[df1['frame'] == frame]
    df2_temp = df2.loc[df2['frame'] == frame]
    for cur_id in list(df1_temp['id']):
        df1_tt = df1_temp.loc[df1_temp['id'] == cur_id]
#         df2_tt = df2_temp.loc[df2_temp['id'] == cur_id]
        x_1 = float(df1_tt['x'])
        y_1 = float(df1_tt['y'])
        match_id = most_common_dict_v2[cur_id]
#         print(cur_id,"cur",(match_id),"matchID")
        if match_id in list(df2_temp['id']):
#             print(cur_id,match_id, "pair")
            df2_tt = df2_temp.loc[df2_temp['id'] == match_id]
            x_2 = float(df2_tt['x'])
            y_2 = float(df2_tt['y'])
#             print(x_1,y_1,cur_id,l_id_corrected,match_id,x_2,y_2) 
            left_cor = np.array([x_1,y_1])
            right_cor = np.array([x_2,y_2])
            
            undistortedPoints1 = cv2.undistortPoints(left_cor, cameraMatrix1, distCoeffs1, P=cameraMatrix1)  # result Nx1x2. No need for np.expand_dims(imagePoints1, axis=1)
            undistortedPoints2 = cv2.undistortPoints(right_cor, cameraMatrix2, distCoeffs2, P=cameraMatrix2)
    
            # Perform triangulation
            point_3d_homogeneous = cv2.triangulatePoints(P1, P2, undistortedPoints1, undistortedPoints2)
            # Convert homogeneous coordinates to Cartesian
            point_3d_cartesian = point_3d_homogeneous[:3] / point_3d_homogeneous[3]
            # Extract x, y, z coordinates
            x, y, z = point_3d_cartesian.flatten()

        #     new_row_data = {'left_id': left_id, 'right_id': row['id'],'difference': difference,'frame':row['frame']}
        #     new_row = pd.DataFrame(new_row_data,index=[0])

            df_temp = pd.DataFrame({'frame':frame,'left_id': cur_id, 'right_id': match_id,'x': x,'y': y, 'z': z},index=[0])
            df_3d_v3 = pd.concat([df_3d_v3, df_temp], ignore_index=True)
            
display(df_3d_v3)


/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_6241/4105640371.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  x_1 = float(df1_tt['x'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_6241/4105640371.py:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y_1 = float(df1_tt['y'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_6241/4105640371.py:16: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  x_2 = float(df2_tt['x'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_6241/4105640371.py:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y_2 = float(df2_tt

,frame,left_id,right_id,x,y,z
0,1,0,5.0,475.641316,-251.609266,2175.999486
1,1,1,6.0,358.941668,-255.299200,2170.809067
2,1,2,3.0,1187.891829,-270.103752,2587.299296
3,1,3,0.0,273.150068,-478.047934,2606.051411
4,1,4,2.0,634.803972,-551.119175,2809.724279
...,...,...,...,...,...,...
30174,3117,4,2.0,116.005151,-677.619052,2495.082079
30175,3117,5,1.0,770.093280,-740.882905,2565.283206
30176,3117,6,7.0,26.176099,-131.034777,2496.313876
30177,3117,7,8.0,153.822302,-587.537830,2244.230237


In [21]:
df_3d_v2.to_csv('3d_coordinatesV2.csv', index=False)
df_3d_v3.to_csv('3d_coordinatesV3.csv', index=False)

In [22]:
df_3d_v2_ground_truth = pd.DataFrame()
for frame in set(df1['frame']):
#     print(frame,"frame")
    df1_temp = df1.loc[df1['frame'] == frame]
    df2_temp = df2.loc[df2['frame'] == frame]
    for cur_id in list(df1_temp['id']):
        df1_tt = df1_temp.loc[df1_temp['id'] == cur_id]
#         df2_tt = df2_temp.loc[df2_temp['id'] == cur_id]
        x_1 = float(df1_tt['x'])
        y_1 = float(df1_tt['y'])
        match_id = gt_matches[cur_id]
#         print(cur_id,"cur",(match_id),"matchID")
        if match_id in list(df2_temp['id']):
#             print(cur_id,match_id, "pair")
            df2_tt = df2_temp.loc[df2_temp['id'] == match_id]
            x_2 = float(df2_tt['x'])
            y_2 = float(df2_tt['y'])
#             print(x_1,y_1,cur_id,l_id_corrected,match_id,x_2,y_2) 
            left_cor = np.array([x_1,y_1])
            right_cor = np.array([x_2,y_2])
            
            undistortedPoints1 = cv2.undistortPoints(left_cor, cameraMatrix1, distCoeffs1, P=cameraMatrix1)  # result Nx1x2. No need for np.expand_dims(imagePoints1, axis=1)
            undistortedPoints2 = cv2.undistortPoints(right_cor, cameraMatrix2, distCoeffs2, P=cameraMatrix2)
    
            # Perform triangulation
            point_3d_homogeneous = cv2.triangulatePoints(P1, P2, undistortedPoints1, undistortedPoints2)
            # Convert homogeneous coordinates to Cartesian
            point_3d_cartesian = point_3d_homogeneous[:3] / point_3d_homogeneous[3]
            # Extract x, y, z coordinates
            x, y, z = point_3d_cartesian.flatten()

        #     new_row_data = {'left_id': left_id, 'right_id': row['id'],'difference': difference,'frame':row['frame']}
        #     new_row = pd.DataFrame(new_row_data,index=[0])

            df_temp = pd.DataFrame({'frame':frame,'left_id': cur_id, 'right_id': match_id,'x': x,'y': y, 'z': z},index=[0])
            df_3d_v2_ground_truth = pd.concat([df_3d_v2_ground_truth, df_temp], ignore_index=True)
            
display(df_3d_v2_ground_truth)

/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_6241/26377018.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  x_1 = float(df1_tt['x'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_6241/26377018.py:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y_1 = float(df1_tt['y'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_6241/26377018.py:16: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  x_2 = float(df2_tt['x'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_6241/26377018.py:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y_2 = float(df2_tt['y'])


,frame,left_id,right_id,x,y,z
0,1,0,5,475.641316,-251.609266,2175.999486
1,1,1,6,358.941668,-255.299200,2170.809067
2,1,2,3,1187.891829,-270.103752,2587.299296
3,1,3,0,273.150068,-478.047934,2606.051411
4,1,4,2,634.803972,-551.119175,2809.724279
...,...,...,...,...,...,...
30173,3117,4,2,116.005151,-677.619052,2495.082079
30174,3117,5,1,770.093280,-740.882905,2565.283206
30175,3117,6,7,26.176099,-131.034777,2496.313876
30176,3117,7,8,153.822302,-587.537830,2244.230237


In [48]:
# display(df_3d_v2_ground_truth.loc[df_3d_v2_ground_truth['frame'] == 902])
display(df1.loc[df1['frame'] == 902])
display(df2.loc[df2['frame'] == 902])
gt_matches

,frame,id,x,y,x_offset,y_offset
7524,902,0,1178.59,438.66,35.47,17.04
7525,902,1,1138.23,347.37,17.30,36.58
7526,902,3,1010.83,293.09,16.92,11.65
7527,902,4,1162.66,293.15,24.05,8.23
7528,902,5,1284.21,369.36,20.89,10.81
7529,902,6,1118.44,447.27,19.57,11.71
7530,902,7,1036.36,339.11,24.82,22.00
7531,902,8,1173.55,573.36,23.40,36.92


,frame,id,x,y,x_offset,y_offset
7542,902,0,933.28,313.01,23.47,8.20
7543,902,1,1218.38,391.57,20.09,14.30
7544,902,2,1095.50,310.88,26.49,12.19
7545,902,4,1057.97,614.87,22.93,23.90
7546,902,5,1078.61,464.07,27.20,16.79
7547,902,6,1023.64,372.89,19.82,32.39
7548,902,7,1030.24,467.38,21.44,17.68
7549,902,8,909.45,362.39,31.40,24.30


{3: 0,
 5: 1,
 4: 2,
 2: 3,
 8: 4,
 0: 5,
 1: 6,
 6: 7,
 7: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14}

In [23]:
display(df_3d_v3.loc[df_3d_v3['frame'] == 902])
display(df_3d_v2_ground_truth.loc[df_3d_v2_ground_truth['frame'] == 902])

,frame,left_id,right_id,x,y,z
7524,902,0,5.0,430.456707,-220.883530,2279.179032
7525,902,1,6.0,332.067278,-372.426865,2147.684775
7526,902,3,0.0,115.051424,-556.319108,2516.037193
7527,902,4,2.0,454.195682,-574.422110,2582.304252
7528,902,5,1.0,710.191445,-397.854767,2530.973974
7529,902,6,7.0,332.498691,-220.994933,2423.638395
7530,902,7,8.0,139.824897,-375.059377,2073.082995
7531,902,8,4.0,401.901962,48.028423,2177.117915


,frame,left_id,right_id,x,y,z
7524,902,0,5,430.456707,-220.883530,2279.179032
7525,902,1,6,332.067278,-372.426865,2147.684775
7526,902,3,0,115.051424,-556.319108,2516.037193
7527,902,4,2,454.195682,-574.422110,2582.304252
7528,902,5,1,710.191445,-397.854767,2530.973974
7529,902,6,7,332.498691,-220.994933,2423.638395
7530,902,7,8,139.824897,-375.059377,2073.082995
7531,902,8,4,401.901962,48.028423,2177.117915


In [24]:
# get performance overall
performance_dict = dict()
for frame in set(df_3d_v2_ground_truth['frame']):
    temp_df_pred = df_3d_v2.loc[df_3d_v2['frame'] == frame]
    temp_df_truth = df_3d_v2_ground_truth.loc[df_3d_v2_ground_truth['frame'] == frame]
    predicted_matches_per_frame = 0
    wrong_predicted_matches_per_frame = 0
    for id_p in list(temp_df_pred['left_id']):
        if float(temp_df_pred.loc[temp_df_pred['left_id'] == id_p]['right_id']) == gt_matches[id_p]:
            predicted_matches_per_frame += 1
        else:
            wrong_predicted_matches_per_frame += 1
    not_guessed_at_all = len(list(temp_df_truth['left_id'])) - len(list(temp_df_pred['left_id']))
    performance_sub_dict = dict()
    performance_sub_dict['amount_correctly_predicted'] = predicted_matches_per_frame
    performance_sub_dict['amount_wrongly_predicted'] = wrong_predicted_matches_per_frame
    performance_sub_dict['amount_not_predicted'] = not_guessed_at_all
    performance_sub_dict['amount_fish'] = len(temp_df_truth['left_id'])
    performance_dict[frame] = performance_sub_dict
    
performance_dict

NameError: name 'df_3d_v2' is not defined

In [25]:
# get performance overall V2 epi per frame
performance_dict_v2 = dict()
for frame in set(df_3d_v2_ground_truth['frame']):
    temp_df_pred = df_3d_v3.loc[df_3d_v3['frame'] == frame]
    temp_df_truth = df_3d_v2_ground_truth.loc[df_3d_v2_ground_truth['frame'] == frame]
    predicted_matches_per_frame = 0
    wrong_predicted_matches_per_frame = 0
    for id_p in list(temp_df_pred['left_id']):
        if float(temp_df_pred.loc[temp_df_pred['left_id'] == id_p]['right_id']) == gt_matches[id_p]:
            predicted_matches_per_frame += 1
        else:
            wrong_predicted_matches_per_frame += 1
    not_guessed_at_all = len(list(temp_df_truth['left_id'])) - len(list(temp_df_pred['left_id']))
    performance_sub_dict = dict()
    performance_sub_dict['amount_correctly_predicted'] = predicted_matches_per_frame
    performance_sub_dict['amount_wrongly_predicted'] = wrong_predicted_matches_per_frame
    performance_sub_dict['amount_not_predicted'] = not_guessed_at_all
    performance_sub_dict['amount_fish'] = len(temp_df_truth['left_id'])
    performance_dict_v2[frame] = performance_sub_dict
    
performance_dict_v2

/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_6241/4114858861.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if float(temp_df_pred.loc[temp_df_pred['left_id'] == id_p]['right_id']) == gt_matches[id_p]:


{1: {'amount_correctly_predicted': 9,
  'amount_wrongly_predicted': 0,
  'amount_not_predicted': 0,
  'amount_fish': 9},
 2: {'amount_correctly_predicted': 9,
  'amount_wrongly_predicted': 0,
  'amount_not_predicted': 0,
  'amount_fish': 9},
 3: {'amount_correctly_predicted': 9,
  'amount_wrongly_predicted': 0,
  'amount_not_predicted': 0,
  'amount_fish': 9},
 4: {'amount_correctly_predicted': 9,
  'amount_wrongly_predicted': 0,
  'amount_not_predicted': 0,
  'amount_fish': 9},
 5: {'amount_correctly_predicted': 9,
  'amount_wrongly_predicted': 0,
  'amount_not_predicted': 0,
  'amount_fish': 9},
 6: {'amount_correctly_predicted': 9,
  'amount_wrongly_predicted': 0,
  'amount_not_predicted': 0,
  'amount_fish': 9},
 7: {'amount_correctly_predicted': 9,
  'amount_wrongly_predicted': 0,
  'amount_not_predicted': 0,
  'amount_fish': 9},
 8: {'amount_correctly_predicted': 9,
  'amount_wrongly_predicted': 0,
  'amount_not_predicted': 0,
  'amount_fish': 9},
 9: {'amount_correctly_predicted

In [66]:
total_amount_predicted_correct = sum(item['amount_correctly_predicted'] for item in performance_dict.values())
total_amount_predicted_wrong = sum(item['amount_wrongly_predicted'] for item in performance_dict.values())
total_amount_missed_fish = sum(item['amount_not_predicted'] for item in performance_dict.values())
percentage_guessed_correct = total_amount_predicted_correct / (total_amount_predicted_correct + total_amount_predicted_wrong)
percentage_fish_missed = total_amount_missed_fish / (total_amount_predicted_correct + total_amount_predicted_wrong)

print(total_amount_predicted_correct,"total_amount_predicted_correct")
print(total_amount_predicted_wrong,"total_amount_predicted_wrong")
print(total_amount_missed_fish,"total_amount_missed_fish")
print(percentage_guessed_correct,"percentage_guessed_correct")
print(percentage_fish_missed ,"percentage_fish_missed ")

26018 total_amount_predicted_correct
2113 total_amount_predicted_wrong
2047 total_amount_missed_fish
0.9248871351889375 percentage_guessed_correct
0.07276669865984145 percentage_fish_missed 


In [37]:
for key, value in performance_dict_v2.items():
    if value['amount_not_predicted'] != 0:
        print(key, value['amount_not_predicted'], value['amount_wrongly_predicted'], value['amount_correctly_predicted'])

display(df_3d_v3.loc[df_3d_v3['frame'] == 2917])
display(df_3d_v2_ground_truth.loc[df_3d_v2_ground_truth['frame'] == 2917])
display(df1.loc[df1['frame']==2917])
display(df2.loc[df2['frame']==2917])

2917 -1 1 12


,frame,left_id,right_id,x,y,z
27566,2917,10,10.0,275.030826,-606.091172,2523.579885
27567,2917,12,12.0,509.015350,-747.528649,2478.047614
27568,2917,11,11.0,417.922921,-470.682561,2584.307000
27569,2917,13,13.0,896.983583,168.700898,2691.605371
27570,2917,14,13.0,2340.322954,964.584449,11370.140296
27571,2917,0,5.0,444.175354,-425.844706,2114.161407
27572,2917,1,6.0,684.389737,-656.175152,2242.166403
27573,2917,3,0.0,166.374465,-361.030129,2433.124058
27574,2917,4,2.0,100.224582,-608.297025,2488.298561
27575,2917,5,1.0,749.844265,-673.952944,2498.489722


,frame,left_id,right_id,x,y,z
27566,2917,10,10,275.030826,-606.091172,2523.579885
27567,2917,12,12,509.015350,-747.528649,2478.047614
27568,2917,11,11,417.922921,-470.682561,2584.307000
27569,2917,13,13,896.983583,168.700898,2691.605371
27570,2917,0,5,444.175354,-425.844706,2114.161407
27571,2917,1,6,684.389737,-656.175152,2242.166403
27572,2917,3,0,166.374465,-361.030129,2433.124058
27573,2917,4,2,100.224582,-608.297025,2488.298561
27574,2917,5,1,749.844265,-673.952944,2498.489722
27575,2917,6,7,58.172247,-140.323493,2445.429934


,frame,id,x,y,x_offset,y_offset
27588,2917,10,1084.66,271.21,12.06,33.39
27589,2917,12,1195.98,202.77,16.10,20.79
27590,2917,11,1146.49,338.95,14.29,8.89
27591,2917,13,1345.10,624.34,36.50,21.46
27592,2917,14,1199.26,665.90,18.63,14.34
27593,2917,0,1202.46,316.30,22.83,39.99
27594,2917,1,1310.69,213.32,12.98,32.28
27595,2917,3,1037.64,380.20,16.06,8.43
27596,2917,4,1004.41,266.88,24.39,10.59
27597,2917,5,1305.20,240.11,11.32,30.55


,frame,id,x,y,x_offset,y_offset
28024,2917,12,1124.03,216.12,18.30,28.00
28025,2917,10,1009.97,290.56,11.29,30.83
28026,2917,13,1291.53,664.86,44.11,21.67
28027,2917,11,1077.40,360.00,14.02,10.40
28028,2917,0,950.05,397.23,24.40,14.00
28029,2917,1,1241.72,254.99,12.91,29.24
28030,2917,2,925.12,284.70,28.99,12.81
28031,2917,4,993.24,605.44,28.61,24.06
28032,2917,5,1087.93,338.60,18.90,36.94
28033,2917,6,1222.55,228.18,19.95,38.37


In [26]:
total_amount_predicted_correct = sum(item['amount_correctly_predicted'] for item in performance_dict_v2.values())
total_amount_predicted_wrong = sum(item['amount_wrongly_predicted'] for item in performance_dict_v2.values())
total_amount_missed_fish = sum(item['amount_not_predicted'] for item in performance_dict_v2.values())
percentage_guessed_correct = total_amount_predicted_correct / (total_amount_predicted_correct + total_amount_predicted_wrong)
percentage_fish_missed = total_amount_missed_fish / (total_amount_predicted_correct + total_amount_predicted_wrong)

print(total_amount_predicted_correct,"total_amount_predicted_correct")
print(total_amount_predicted_wrong,"total_amount_predicted_wrong")
print(total_amount_missed_fish,"total_amount_missed_fish")
print(percentage_guessed_correct,"percentage_guessed_correct")
print(percentage_fish_missed ,"percentage_fish_missed ")

29978 total_amount_predicted_correct
201 total_amount_predicted_wrong
-1 total_amount_missed_fish
0.9933397395539945 percentage_guessed_correct
-3.31356241094801e-05 percentage_fish_missed 


In [41]:
display(df_3d_v2_ground_truth)
df_to_analyze = df_3d_v2_ground_truth.drop('right_id',axis=1)
df_to_analyze = df_to_analyze.rename(columns={'left_id': 'fish_id'})
display(df_to_analyze)
df_to_analyze.to_csv('df_to_analyze.csv', index=False)

,frame,left_id,right_id,x,y,z
0,1,0,5,475.641316,-251.609266,2175.999486
1,1,1,6,358.941668,-255.299200,2170.809067
2,1,2,3,1187.891829,-270.103752,2587.299296
3,1,3,0,273.150068,-478.047934,2606.051411
4,1,4,2,634.803972,-551.119175,2809.724279
...,...,...,...,...,...,...
30173,3117,4,2,116.005151,-677.619052,2495.082079
30174,3117,5,1,770.093280,-740.882905,2565.283206
30175,3117,6,7,26.176099,-131.034777,2496.313876
30176,3117,7,8,153.822302,-587.537830,2244.230237


,frame,fish_id,x,y,z
0,1,0,475.641316,-251.609266,2175.999486
1,1,1,358.941668,-255.299200,2170.809067
2,1,2,1187.891829,-270.103752,2587.299296
3,1,3,273.150068,-478.047934,2606.051411
4,1,4,634.803972,-551.119175,2809.724279
...,...,...,...,...,...
30173,3117,4,116.005151,-677.619052,2495.082079
30174,3117,5,770.093280,-740.882905,2565.283206
30175,3117,6,26.176099,-131.034777,2496.313876
30176,3117,7,153.822302,-587.537830,2244.230237


left id is chosen as new id

In [136]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display

# Assuming df_3d_v2 is already defined
df = df_3d_v2

# Group the DataFrame by 'frame'
grouped = df.groupby('frame')

# Create a function to update the plot based on the selected frame
def update_plot(frame):
    data = grouped.get_group(frame)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(data['x'], data['y'], data['z'], marker='o')
    
    # Annotate each point with its corresponding id
    for i, txt in enumerate(data['left_id']):
        ax.text(data['x'].iloc[i], data['y'].iloc[i], data['z'].iloc[i], str(txt), color='red')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'Frame {frame}')
    plt.show()

# Create a slider widget
frame_slider = widgets.IntSlider(min=df['frame'].min(), max=df['frame'].max(), value=df['frame'].min(), description='Frame')

# Display the slider and plot
widgets.interact(update_plot, frame=frame_slider)


interactive(children=(IntSlider(value=1, description='Frame', max=3117, min=1), Output()), _dom_classes=('widg…

<function __main__.update_plot(frame)>

In [138]:
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
df = df_3d_v2
# Directory to save images
save_dir = 'animation_frames'
os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Group the DataFrame by 'frame'
grouped = df.groupby('frame')

# Define a function to save each frame as an image
def save_frame(frame):
    data = grouped.get_group(frame)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(data['x'], data['y'], data['z'], marker='o')
    
    # Annotate each point with its corresponding id
    for i, txt in enumerate(data['left_id']):
        ax.text(data['x'].iloc[i], data['y'].iloc[i], data['z'].iloc[i], str(txt), color='red')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'Frame {frame}')
    
    # Save the figure
    save_path = os.path.join(save_dir, f'frame_{frame}.png')
    plt.savefig(save_path)
    plt.close(fig)  # Close the figure to release memory

# Loop through each frame and save it
for frame in df['frame'].unique():
    save_frame(frame)


In [140]:
import cv2
import os

# Define the directory containing the frames
frames_dir = 'animation_frames'

# Define the output video file name
output_video = 'animationV2.mp4'

# Create a list to hold the paths of all frame images
frame_paths = [os.path.join(frames_dir, f) for f in os.listdir(frames_dir) if f.endswith('.png')]

# Sort the frame paths by file name
frame_paths.sort()

# Read the first frame to get dimensions
first_frame = cv2.imread(frame_paths[0])
height, width, _ = first_frame.shape

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, 30, (width, height))  # Adjust FPS as needed

# Write frames to video
for frame_path in frame_paths:
    frame = cv2.imread(frame_path)
    out.write(frame)

# Release video writer
out.release()

print("Video generation complete!")


Video generation complete!
